<a href="https://colab.research.google.com/github/colesmcintosh/aws-knowledge-base-rag/blob/main/aws_knowledge_base_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Querying a Knowledge Base with AWS Bedrock

This notebook demonstrates how to query an AWS Bedrock Knowledge Base using the `RetrieveAndGenerate` API.

## Prerequisites
- Install Boto3: `pip install boto3`
- Set up AWS credentials (e.g., via the AWS CLI or environment variables).
- Have an AWS Bedrock Knowledge Base configured.
- Replace placeholder values with your actual AWS identifiers (e.g., `knowledge_base_id`, `model_arn`).

## Overview
- Initialize the AWS Bedrock Runtime client.
- Retrieve the list of available foundation models.
- Send a query to the Knowledge Base.
- Parse and display the response in a readable format.


In [ ]:
!pip install boto3

In [2]:
import boto3
import json
from google.colab import userdata

## Initialize the AWS Bedrock Client

Create a Boto3 client for AWS Bedrock. Adjust the region as needed.

In [3]:
# Prompt the user for credentials if not set in environment variables
aws_access_key_id = userdata.get("AWS_ACCESS_KEY") or input("AWS_ACCESS_KEY: ")
aws_secret_access_key = userdata.get("AWS_SECRET_KEY") or input("AWS_SECRET_KEY: ")
aws_knowledge_base_id = userdata.get("AWS_KNOWLEDGE_BASE_ID") or input("AWS_KNOWLEDGE_BASE_ID: ")

# Initialize the Bedrock Runtime client
runtime_client = boto3.client(
    'bedrock-agent-runtime',
    region_name='us-west-2',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Initialize the Bedrock client to list foundation models
client = boto3.client(
    'bedrock',
    region_name='us-west-2',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

## Retrieve Available Foundation Models

Fetch the list of available foundation models in AWS Bedrock.

In [ ]:
models = client.list_foundation_models()["modelSummaries"]
json.dumps(models, indent=4)

## Define Query Parameters

Set up the knowledge base ID, model ARN, and the question to query.

In [5]:

question = input("Question: ")
model_arn = userdata.get("AWS_MODEL_ARN") or input("AWS_MODEL_ARN: ")  # User must provide the model ARN

# Construct the payload for the RetrieveAndGenerate API call
payload = {
    "input": {
        "text": question
    },
    "retrieveAndGenerateConfiguration": {
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": aws_knowledge_base_id,
            "modelArn": model_arn
        },
        "type": "KNOWLEDGE_BASE"
    }
}

Question: What does Cole specialize in?


## Query the AWS Bedrock Knowledge Base

Send a request to retrieve and generate a response based on the knowledge base.

In [ ]:

try:
    response = runtime_client.retrieve_and_generate(
        input=payload["input"],
        retrieveAndGenerateConfiguration=payload["retrieveAndGenerateConfiguration"]
    )
    print("Raw API Response:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print("Error during the RetrieveAndGenerate API call:")
    print(e)

## Parse and Display Results

This function formats and displays the generated response along with source citations.

In [7]:

def display_result(result):
    """Formats and prints the generated response and citations."""
    output_text = result.get("output", {}).get("text", "No response generated.")
    session_id = result.get("sessionId", "N/A")

    print("\n==== Generated Answer ====")
    print(output_text)

    citations = result.get("citations", [])
    if not citations:
        print("\n==== Citations ====")
        print("No citations found.")
    else:
        print("\n==== Citations ====")
        for idx, citation in enumerate(citations, start=1):
            gen_text = citation.get("generatedResponsePart", {}).get("textResponsePart", {}).get("text", "")
            print(f"\nCitation {idx}: {gen_text}")

            references = citation.get("retrievedReferences", [])
            if not references:
                print("  No retrieved references available.")
            else:
                for ref in references:
                    content_text = ref.get("content", {}).get("text", "")
                    source_url = ref.get("location", {}).get("webLocation", {}).get("url", "N/A")
                    excerpt = content_text[:200] + ("..." if len(content_text) > 200 else "")
                    print(f"  - Source: {source_url}")
                    print(f"    Excerpt: {excerpt}")

    print("\n==== Session ID ====")
    print(session_id)

# Call the function to display results
display_result(response)


==== Generated Answer ====
Cole McIntosh specializes in developing cutting-edge AI systems and data solutions, with a focus on leveraging Large Language Models (LLMs) and AI agents to build intelligent, autonomous systems for complex decision-making and problem-solving.

==== Citations ====

Citation 1: Cole McIntosh specializes in developing cutting-edge AI systems and data solutions, with a focus on leveraging Large Language Models (LLMs) and AI agents to build intelligent, autonomous systems for complex decision-making and problem-solving.
  - Source: https://www.colemcintosh.io/docs/cole_mcintosh_cv.pdf
    Excerpt: Cole McIntosh     AI and Data Engineer     Foreword     I am an innovative AI and Data Engineer specializing in developing cutting-edge AI systems and data solutions. With expertise in Large Language ...
  - Source: https://www.colemcintosh.io
    Excerpt: Cole McIntosh ### About Me I am an innovative AI and Data Engineer based in Denver, Colorado, specializing in deve

# Conclusion

In this notebook, we successfully:
- Queried an AWS Bedrock Knowledge Base.
- Retrieved a generated response.
- Displayed citations from retrieved knowledge sources.

**Next Steps:**
- Enhance queries by refining the model's retrieval and generation configurations.
- Implement session management to maintain conversational context.
- Optimize knowledge base indexing for improved responses.

For more details, refer to the [AWS Bedrock Guide](https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html).